WE should use percision score since we care about mostly identifying who has pneumonia

# Preparation

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.layers import average 
from keras.models import Input, Model
from sklearn import metrics as mt
from matplotlib import pyplot as plt
from skimage.io import imshow
import seaborn as sns
keras.__version__

Using TensorFlow backend.


'2.3.1'

In [105]:
pneumonia_images = []
healthy_images = []
import os
import struct
import numpy as np
import glob
import cv2
images = glob.glob("chest_xray/test/NORMAL/*")
for image in images:
    im = cv2.imread(image,0)
    im = cv2.resize(im,(300,300))
    im = im.flatten()/255 -.5
    healthy_images.append(im)
images = glob.glob("chest_xray/test/PNEUMONIA/*")
for image in images:
    penumonia_images
    im = cv2.imread(image,0)
    im = cv2.resize(im,(300,300))
    im = im.flatten()/255 -.5
    pneumonia_images.append(im)
pneumonia_images 


[array([ 0.19411765,  0.24117647,  0.26078431, ..., -0.40196078,
        -0.40588235, -0.40588235]),
 array([ 0.03333333, -0.01372549, -0.00588235, ..., -0.4254902 ,
        -0.4254902 , -0.4254902 ]),
 array([ 0.3745098 ,  0.34705882,  0.37058824, ..., -0.44117647,
        -0.44117647, -0.44117647]),
 array([ 0.0254902 ,  0.00196078, -0.04509804, ..., -0.40588235,
        -0.40980392, -0.40980392]),
 array([-0.4372549 , -0.4254902 , -0.40196078, ..., -0.49607843,
        -0.49607843, -0.49607843]),
 array([-0.5, -0.5, -0.5, ..., -0.5, -0.5, -0.5]),
 array([ 0.00588235,  0.0254902 ,  0.0254902 , ..., -0.4254902 ,
        -0.4254902 , -0.4254902 ]),
 array([-0.5, -0.5, -0.5, ..., -0.5, -0.5, -0.5]),
 array([-0.31960784, -0.31568627, -0.30784314, ..., -0.39019608,
        -0.39411765, -0.39411765]),
 array([-0.37058824, -0.35882353, -0.34313725, ..., -0.41764706,
        -0.41764706, -0.41764706]),
 array([-0.24117647, -0.04509804,  0.02941176, ..., -0.5       ,
        -0.5       , -0.5

In [59]:
#Now lets put them together with y values
healthy_images
b = np.zeros((len(healthy_images),1))
a = np.ones((len(pneumonia_images),1))
healthy_images = np.hstack((healthy_images, b))
pneumonia_images = np.hstack((pneumonia_images, a))
images = np.vstack([healthy_images, pneumonia_images])

In [101]:
y = images[:,-1]
X = images[:,:-1]

# folding below
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)
img_wh = 300
y_test_ohe = keras.utils.to_categorical(y_test, 2)
y_train_ohe = keras.utils.to_categorical(y_train, 2)
X_train_img = np.expand_dims(X_train.reshape((-1,img_wh,img_wh)), axis=3)
X_test_img = np.expand_dims(X_test.reshape((-1,img_wh,img_wh)), axis=3)
folds = StratifiedKFold(n_splits=10,random_state=1).split(X_train, y_train)

In [102]:
y_train_ohe

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.

# Modeling

In [98]:
y_test

array([1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1.,
       0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0.,
       1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0.,
       0., 0., 1., 1., 1., 1.])

In [35]:

#this is expansion on the dataset. This is a slower way to do it. When we use a keras generator we call .fit on a python generator
#this will yield batches

datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False, #do we want to make it 0 mean
    featurewise_std_normalization=False, #do we take the whole batch and make it 0 mean, no
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=5, # used, Int. Degree range for random rotations. Randomly rotate images 5 degrees 
    width_shift_range=0.1, # used, Float (fraction of total width). Range for random horizontal shifts. 
    height_shift_range=0.1, # used,  Float (fraction of total height). Range for random vertical shifts.
    shear_range=0., # Float. Shear Intensity (Shear angle in counter-clockwise direction as radians)
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=True,
    vertical_flip=False, 
    rescale=None) #this generator will esentially run forever. This will manipulate our data, will give us different datasets every time


In [106]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
np.random.seed(1000)
#Instantiate an empty model
#get a bigger dataset
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=32, input_shape=(300,300,1), kernel_size=(14,14),  padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=64, kernel_size=(7,7), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid', data_format="channels_last"))

# 3rd Convolutional Layer
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='valid', data_format="channels_last"))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid', data_format="channels_last"))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='valid', data_format="channels_last"))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# 3rd Fully Connected Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 287, 287, 32)      6304      
_________________________________________________________________
activation_10 (Activation)   (None, 287, 287, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 143, 143, 32)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 137, 137, 64)      100416    
_________________________________________________________________
activation_11 (Activation)   (None, 137, 137, 64)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 68, 68, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 66, 66, 128)      

In [108]:
scores = []
#datagen.fit(X_train)
print(folds)
for k, (train, test) in enumerate(folds):
    print("new fold")
    datagen.fit(X_train_img)
    model.fit_generator(datagen.flow(X_train_img[train], y_train_ohe[train], batch_size=32),
             steps_per_epoch=int(len(X_train[train])/128),
             epochs=5, verbose=1,
             validation_data=(X_test_img,y_test_ohe),
             callbacks=[EarlyStopping(monitor='val_loss', patience=2)]
             )


<generator object _BaseKFold.split at 0x1c558dfa98>
new fold
Epoch 1/5


KeyboardInterrupt: 

In [87]:
print("SDS")
for k, (train, test) in enumerate(folds):
    print(len(X_train))
    print(train)

SDS
499
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 151 152 155 197 199 200 202 203 205 206 207 208 209 210
 211 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229
 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247
 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265
 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283
 284 285 286 287 288 289 290 291 292 293 29